# Basic Usage

This page covers common usage patterns and best practices when working with QOBLIB in OMMX format. Please refer [OMMX focumentation](https://jij-inc.github.io/ommx/en/introduction.html) as well.

## Dataset Instantiation

All dataset classes follow the same pattern:

In [ ]:
from ommx_quantum_benchmarks.qoblib import Marketsplit

# Create dataset instance
marketsplit = Marketsplit()

## Accessing Dataset Properties

In [ ]:
dataset = Marketsplit()

# Basic information
print(f"Dataset ID: {dataset.name}")
print(f"Description: {dataset.description}")
print(f"Available models: {dataset.model_names}")

# Container registry information
print(f"Base URL: {dataset.base_url}")

## Instance Management

### Listing Available Instances

In [ ]:
# Get all instances for all models
for model_name, instances in dataset.available_instances.items():
    print(f"Model '{model_name}': {len(instances)} instances")
    
# Get instances for specific model
binary_linear_instances = dataset.available_instances["binary_linear"]
print(f"Binary linear instances: {binary_linear_instances[:5]}")  # First 5

### Loading Specific Instances

In [ ]:
# Load instance and solution
model_name = "binary_linear"
instance_name = "ms_03_050_002"

instance, solution = dataset(model_name, instance_name)
print("Successfully loaded instance and solution")

## Working with Solutions

### Solution Validation

In [ ]:
if solution is not None:
    # Evaluate solution using the instance
    evaluated = instance.evaluate(solution.state)
    
    # Compare original and evaluated solutions
    obj_match = solution.objective == evaluated.objective
    feas_match = solution.feasible == evaluated.feasible
    state_match = solution.state.entries == evaluated.state.entries
    
    print(f"Objective values match: {obj_match}")
    print(f"Feasibility matches: {feas_match}")
    print(f"State entries match: {state_match}")

### Accessing Solution Data

In [ ]:
if solution is not None:
    print(f"Objective value: {solution.objective}")
    print(f"Is feasible: {solution.feasible}")
    print(f"Number of variables: {len(solution.state.entries)}")
    
    # Access variable assignments
    for id, entry in solution.state.entries.items():
        print(f"Variable {id}: {entry}")

## Note about Annotations
The downloaded instance includes various annotations accessible via the `annotations` property.

| Annotation | Property | Description |
| --- | --- | --- |
| `org.ommx.v1.instance.title` | `title` | The name of the instance |
| `org.ommx.v1.instance.license` | `license` | The license of the dataset |
| `org.ommx.v1.instance.dataset` | `dataset` | The name of the dataset to which this instance belongs |
| `org.ommx.v1.instance.authors` | `authors` | The authors of the instnce |
| `org.ommx.v1.instance.variables` | `num_variables` | The total nuimber of decision variables in the instance |
| `org.ommx.v1.instance.constraints` | `num_constraints` | The total number of constraint conditions in the instance |

So does solution.

| Annotation | Property | Description |
| --- | --- | --- |
| `org.ommx.v1.solution.instance` | `instance` | The digest of the instance to which this solution applies |
| `org.ommx.v1.solution.authors` | `authors` | The authors of the solution |